# RNN Forecasting for IBM Stock

In [ ]:
import sys
import os

In [ ]:
sys.path.append(os.path.abspath("../src"))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from rnn_forecasting import RNNForecaster
from preprocessing import Preprocessor

## 1. Data Preprocessing

In [ ]:
preprocessor = Preprocessor("../data/raw/IBM_Stock_1980_2025.csv")

df = preprocessor.load_data()
df = preprocessor.add_features()

train, test = preprocessor.split(test_size=0.2)

df.tail()

c:\Users\clemm\Documents\Personnal Projects\ibm-stock-prices\src\preprocessing.py:73: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill').fillna(method='ffill')


,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Adj Factor,Daily_Return,Cumulative_Return,SMA_30,SMA_100,EMA_30,EMA_100,Volatility_30d,BB_Mid,BB_Upper,BB_Lower
Date,,,,,,,,,,,,,,,,,,,
2025-07-24,261.250000,262.049988,252.750000,260.510010,260.510010,22647700,0.0,0.0,1.0,-0.076238,268.722515,285.320334,259.747270,280.563528,263.605410,0.017962,285.837503,300.414279,271.260727
2025-07-25,260.019989,260.799988,256.350006,259.720001,259.720001,7758700,0.0,0.0,1.0,-0.003033,267.904569,284.593668,259.859437,279.218785,263.528472,0.017523,284.227003,302.594152,265.859855
2025-07-28,260.299988,264.000000,259.609985,263.209991,263.209991,5192500,0.0,0.0,1.0,0.013438,271.517977,283.999667,259.976507,278.185959,263.522165,0.017764,282.902502,303.314561,262.490444
2025-07-29,264.299988,265.799988,261.019989,262.410004,262.410004,4627300,0.0,0.0,1.0,-0.003039,270.689699,283.506001,260.104052,277.168156,263.500142,0.017632,281.284003,302.832354,259.735651
2025-07-30,261.600006,262.000000,258.899994,260.260010,260.260010,3718300,0.0,0.0,1.0,-0.008193,268.463674,282.787002,260.236517,276.077308,263.435981,0.017323,279.737003,302.684959,256.789047


## 2. LSTM Forecasting

In [ ]:
lstm_forecaster = RNNForecaster(
    model_type="lstm",
    hidden_size=64,
    num_layers=2,
    window_size=20,
    batch_size=32
)
lstm_forecaster.prepare_data(train, test)
lstm_forecaster.train(epochs=50, lr=0.0001)
lstm_predictions = lstm_forecaster.predict()

metrics = lstm_forecaster.evaluate(test.iloc[-len(lstm_predictions):], lstm_predictions)

rmse = metrics["rmse"]
mae = metrics["mae"]
mape = metrics["mape"]

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")


lstm_forecaster.plot_forecast(test.iloc[-len(lstm_predictions):], lstm_predictions, title="LSTM Forecast vs Actual")


Epoch 1/50:   0%|          | 1/5456 [00:00<10:12,  8.90it/s, loss=0.00581]

[Batch 0] X_batch.shape: torch.Size([32, 20, 1]), y_batch.shape: torch.Size([32, 1])
[Batch 0] y_pred min/max: -0.0569/-0.0567
[Batch 0] y_batch min/max: 0.0000/0.0000


Epoch 1/50:   2%|▏         | 105/5456 [00:03<02:23, 37.24it/s, loss=0.0012] 

[Batch 100] X_batch.shape: torch.Size([32, 20, 1]), y_batch.shape: torch.Size([32, 1])
[Batch 100] y_pred min/max: 0.0058/0.0059
[Batch 100] y_batch min/max: 0.0000/0.2141


Epoch 1/50:   4%|▍         | 205/5456 [00:06<02:32, 34.35it/s, loss=0.000881]

...

Epoch 2/50:  44%|████▍     | 2405/5456 [04:09<01:39, 30.68it/s, loss=0.000542]

[Batch 2400] X_batch.shape: torch.Size([32, 20, 1]), y_batch.shape: torch.Size([32, 1])
[Batch 2400] y_pred min/max: -0.0008/0.0829
[Batch 2400] y_batch min/max: 0.0000/0.0593


Epoch 2/50:  46%|████▌     | 2504/5456 [04:13<02:05, 23.51it/s, loss=0.00054] 

[Batch 2500] X_batch.shape: torch.Size([32, 20, 1]), y_batch.shape: torch.Size([32, 1])
[Batch 2500] y_pred min/max: -0.0099/0.0265
[Batch 2500] y_batch min/max: 0.0000/0.0000


KeyboardInterrupt: 

## 2. LightGBM Forecasting

In [ ]:
lgbm_forecaster = RNNForecaster(
    model_type="lstm",
    hidden_size=64,
    num_layers=2,
    window_size=20,
    batch_size=32
)
lgbm_forecaster.prepare_data(train, test)
lgbm_forecaster.train(epochs=50, lr=0.001)
lstm_predictions = lgbm_forecaster.predict()

metrics = lgbm_forecaster.evaluate(test.iloc[-len(lstm_predictions):], lstm_predictions)

rmse = metrics["rmse"]
mae = metrics["mae"]
mape = metrics["mape"]

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")


lgbm_forecaster.plot_forecast(test.iloc[-len(lstm_predictions):], lstm_predictions, title="LSTM Forecast vs Actual")

c:\Users\clemm\Documents\Personnal Projects\ibm-stock-prices\src\preprocessing.py:149: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\b\abs_34s6o8i12c\croot\libtorch_1751464457133\work\torch\csrc\utils\tensor_new.cpp:257.)
  self.X = torch.tensor(self.X, dtype=torch.float32).unsqueeze(-1)


ValueError: LSTM: Expected input to be 2D or 3D, got 4D instead